In [1]:
#import Libraries
import numpy as np 
import matplotlib as plt
import tensorflow as tf 
from tensorflow.keras import layers ,models ,Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator ,load_img ,img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input , decode_predictions,ResNet50
from tensorflow.keras.layers import Conv2D,Flatten,Dense,MaxPool2D,BatchNormalization,GlobalAveragePooling2D

In [2]:
#Putting data into variables
img_height, img_width = (64,64)
batch_size= 32
train_data = f"data/train"
test_data = f"data/test"

In [3]:
#Divide into testing, training and validation
train_data_1 = ImageDataGenerator(preprocessing_function=preprocess_input,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  validation_split=0.4)
train_img = train_data_1.flow_from_directory(train_data,
                                               target_size=(img_height,img_width),
                                               batch_size=batch_size,
                                               class_mode="categorical")
test_img = train_data_1.flow_from_directory(test_data,
                                               target_size=(img_height,img_width),
                                               batch_size=batch_size,
                                               class_mode="categorical")

Found 2637 images belonging to 2 classes.
Found 660 images belonging to 2 classes.


In [4]:
x,y = test_img.next()
x.shape

(32, 64, 64, 3)

In [5]:
#Creating and training neural networks
model_cs = ResNet50(include_top=False,weights="imagenet")
x = model_cs.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024,activation="relu")(x)
pred = Dense(train_img.num_classes,activation='softmax')(x)
model = Model(inputs=model_cs.input,outputs=pred)
for layer in model_cs.layers:
    layer.trainable = False
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(train_img, epochs=10)

Epoch 1/10
83/83 [==============================] - 23s 232ms/step - loss: 1.0372 - accuracy: 0.7960
Epoch 2/10
83/83 [==============================] - 19s 231ms/step - loss: 0.3279 - accuracy: 0.8525
Epoch 3/10
83/83 [==============================] - 20s 243ms/step - loss: 0.2864 - accuracy: 0.8760
Epoch 4/10
83/83 [==============================] - 20s 246ms/step - loss: 0.2784 - accuracy: 0.8779
Epoch 5/10
83/83 [==============================] - 20s 242ms/step - loss: 0.2608 - accuracy: 0.8726
Epoch 6/10
83/83 [==============================] - 21s 252ms/step - loss: 0.2325 - accuracy: 0.8980
Epoch 7/10
83/83 [==============================] - 22s 261ms/step - loss: 0.2236 - accuracy: 0.8995
Epoch 8/10
83/83 [==============================] - 20s 245ms/step - loss: 0.2293 - accuracy: 0.8991
Epoch 9/10
83/83 [==============================] - 25s 298ms/step - loss: 0.2105 - accuracy: 0.9124
Epoch 10/10
83/83 [==============================] - 25s 298ms/step - loss: 0.2147 - accura

In [6]:
model.evaluate(test_img)

21/21 [==============================] - 6s 234ms/step - loss: 0.3509 - accuracy: 0.8455


[0.35093724727630615, 0.8454545736312866]

In [23]:
cnn = Sequential([
                  layers.Conv2D(128, (2,2), activation='relu', input_shape=(64, 64, 3)),
                  layers.MaxPooling2D(3,3),
                  layers.Conv2D(128, kernel_size=(3,3),activation='relu',kernel_initializer='he_normal'),
                  layers.MaxPooling2D(2,2),
                  #layers.Conv2D(64, (2, 2), activation='relu'),
                  layers.Conv2D(128, kernel_size=(2,2),activation='relu',kernel_initializer='he_normal'),
                  layers.MaxPooling2D(2,2),
                  layers.Flatten(),
                  layers.Dense(128,activation='relu'),
                  layers.Dense(2,activation='softmax',kernel_initializer='glorot_normal')
                  
])
cnn.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
cnn.fit(train_img, epochs=10)

Epoch 1/10
83/83 [==============================] - 17s 197ms/step - loss: 2.1788 - accuracy: 0.6959
Epoch 2/10
83/83 [==============================] - 16s 197ms/step - loss: 0.4514 - accuracy: 0.7801
Epoch 3/10
83/83 [==============================] - 18s 213ms/step - loss: 0.3981 - accuracy: 0.8142
Epoch 4/10
83/83 [==============================] - 19s 225ms/step - loss: 0.3918 - accuracy: 0.8153
Epoch 5/10
83/83 [==============================] - 19s 231ms/step - loss: 0.4101 - accuracy: 0.8032
Epoch 6/10
83/83 [==============================] - 18s 210ms/step - loss: 0.3916 - accuracy: 0.8172
Epoch 7/10
83/83 [==============================] - 17s 200ms/step - loss: 0.3630 - accuracy: 0.8237
Epoch 8/10
83/83 [==============================] - 17s 199ms/step - loss: 0.3466 - accuracy: 0.8377
Epoch 9/10
83/83 [==============================] - 17s 201ms/step - loss: 0.3479 - accuracy: 0.8377
Epoch 10/10
83/83 [==============================] - 17s 201ms/step - loss: 0.3375 - accura

In [24]:
cnn.evaluate(test_img)

21/21 [==============================] - 3s 112ms/step - loss: 0.4045 - accuracy: 0.8152


[0.40453600883483887, 0.8151515126228333]

In [27]:

test_image = load_img('3.jpg', target_size = (64, 64))
test_image = img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
#training_set.class_indices
if result[0][0] == 1:
    prediction = 'benign'
else:
    prediction = 'malignant'
prediction

1/1 [==============================] - 0s 18ms/step


'malignant'